In [19]:
import streamlit as st
from watttime import WattTimeMyAccess, WattTimeHistorical, WattTimeForecast


secrets = st.secrets
wt_myaccess = WattTimeMyAccess(st.secrets['WATTTIME_USER'], st.secrets['WATTTIME_PASSWORD'])
wt_hist = WattTimeHistorical(st.secrets['WATTTIME_USER'], st.secrets['WATTTIME_PASSWORD'])
wt_forecast = WattTimeForecast(st.secrets['WATTTIME_USER'], st.secrets['WATTTIME_PASSWORD'])

# return a nested json describing signals and regions you have access to
wt_myaccess.get_access_json()

# return a pandas dataframe describing signals and regions you have access to
wt_myaccess.get_access_pandas()

,signal_type,region,region_name,endpoint,model,data_start,train_start,train_end,type
0,co2_moer,CAISO_NORTH,California ISO Northern,v3/historical,2022-10-01,2020-01-01,2020-03-01,2022-03-01,binned_regression
1,co2_moer,CAISO_NORTH,California ISO Northern,v3/historical,2023-03-01,2020-03-01,2021-01-01,2023-03-01,binned_regression
2,co2_moer,CAISO_NORTH,California ISO Northern,v3/forecast,2022-10-01,2021-12-31,2020-01-01,2021-12-31,NaN
3,co2_moer,CAISO_NORTH,California ISO Northern,v3/forecast,2023-03-01,2023-01-01,2021-01-01,2023-01-01,NaN
4,health_damage,CAISO_NORTH,California ISO Northern,v3/historical,2022-10-01,2020-01-01,2020-03-01,2022-03-01,binned_regression
5,health_damage,CAISO_NORTH,California ISO Northern,v3/forecast,2022-06-03,2019-12-31,2019-01-01,2019-12-31,NaN
6,health_damage,CAISO_NORTH,California ISO Northern,v3/forecast,2022-10-01,2021-12-31,2020-01-01,2021-12-31,NaN


In [20]:
df = wt_hist.get_historical_pandas(
    start='2024-03-01 00:00Z',
    end='2024-06-01 00:00Z',
    region='CAISO_NORTH',

    signal_type='co2_moer', 
)

In [23]:
df

,point_time,value
0,2024-03-01 00:00:00+00:00,102.0
1,2024-03-01 00:05:00+00:00,101.0
2,2024-03-01 00:10:00+00:00,101.0
3,2024-03-01 00:15:00+00:00,101.0
4,2024-03-01 00:20:00+00:00,101.0
...,...,...
26492,2024-05-31 23:40:00+00:00,149.0
26493,2024-05-31 23:45:00+00:00,143.0
26494,2024-05-31 23:50:00+00:00,192.0
26495,2024-05-31 23:55:00+00:00,192.0


In [30]:
df = df[['timestamp', 'value']]
df1 = df.set_index('timestamp').resample('15min').mean()
df1

,value
timestamp,
2024-06-01 00:00:00+00:00,101.333333
2024-06-01 00:15:00+00:00,102.000000
2024-06-01 00:30:00+00:00,1019.666667
2024-06-01 00:45:00+00:00,1018.333333
2024-06-01 01:00:00+00:00,1025.666667
...,...
2024-08-31 23:00:00+00:00,130.000000
2024-08-31 23:15:00+00:00,130.666667
2024-08-31 23:30:00+00:00,140.000000


In [31]:
df1.to_csv('data/emissions.csv')

In [9]:
import pandas as pd

df = pd.read_csv('data/consumed.csv')


In [10]:
df

,date,Read Type,time,kWh
0,2022-04-15T00:00:00Z,kWh Consumed,00:00:00,0.06
1,2022-04-15T00:00:00Z,kWh Consumed,00:15:00,0.07
2,2022-04-15T00:00:00Z,kWh Consumed,00:30:00,0.05
3,2022-04-15T00:00:00Z,kWh Consumed,00:45:00,0.09
4,2022-04-15T00:00:00Z,kWh Consumed,01:00:00,0.08
...,...,...,...,...
72571,2024-05-09T00:00:00Z,kWh Consumed,22:45:00,0.35
72572,2024-05-09T00:00:00Z,kWh Consumed,23:00:00,0.19
72573,2024-05-09T00:00:00Z,kWh Consumed,23:15:00,0.17
72574,2024-05-09T00:00:00Z,kWh Consumed,23:30:00,0.23


In [11]:
df['timestamp'] = pd.to_datetime(df.apply(lambda x: x['date'][:10] + ' ' + x['time'], axis=1))

In [12]:
df = df.rename(columns={'kWh': 'value'})[['timestamp', 'value']]
df

,timestamp,value
0,2022-04-15 00:00:00,0.06
1,2022-04-15 00:15:00,0.07
2,2022-04-15 00:30:00,0.05
3,2022-04-15 00:45:00,0.09
4,2022-04-15 01:00:00,0.08
...,...,...
72571,2024-05-09 22:45:00,0.35
72572,2024-05-09 23:00:00,0.19
72573,2024-05-09 23:15:00,0.17
72574,2024-05-09 23:30:00,0.23


In [13]:
df = df[df.timestamp >= '2024-03-01 00:00:00']

In [16]:
df['timestamp'] = df['timestamp'] + pd.DateOffset(months=1)

/tmp/ipykernel_3335/303065086.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['timestamp'] = df['timestamp'] + pd.DateOffset(months=1)


In [18]:
df.to_csv('data/consumption.csv', index=False)